# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Files to load
file_to_load = "../output_data/cities.csv"
# Read cities file and store into Pandas data frame
cities_data = pd.read_csv(file_to_load)
cities_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,78.58,70,99,16.15,PF,1619286557
1,fort nelson,58.8053,-122.7002,37.40,44,20,6.91,CA,1619286619
2,albany,42.6001,-73.9662,69.01,18,57,5.99,US,1619286342
3,chuy,-33.6971,-53.4616,71.82,69,100,7.20,UY,1619286620
4,zhigansk,66.7697,123.3711,33.19,98,100,10.04,RU,1619286952
...,...,...,...,...,...,...,...,...,...
548,erattupetta,9.7000,76.7833,78.80,83,75,2.35,IN,1619287057
549,pangoa,-12.1167,-73.0000,80.49,72,43,3.15,PE,1619287057
550,port blair,11.6667,92.7500,81.93,75,35,7.11,IN,1619286591
551,parrita,9.5201,-84.3225,82.40,50,20,10.36,CR,1619287057


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Import API for gmaps just in this cell
from api_keys import g_key
import gmaps
gmaps.configure(api_key=g_key)

# Set values
locations = cities_data[["Lat", "Lng"]]
humidity = cities_data["Humidity"]

# Create map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                 max_intensity=300, point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Establish criteria to narrow down the data frame
# Max temp 70-90
# Cloudiness less than 10
# Humidity 30-50

# Create narrowed data frame, dropping rows that don't fit condition
ideal_cities = cities_data[(cities_data["Max Temp"] > 70) & (cities_data["Max Temp"] < 90) &
                           (cities_data["Cloudiness"] < 40) & (cities_data["Humidity"] > 30) &
                           (cities_data["Humidity"] < 50)]
ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,castro,-24.7911,-50.0119,73.76,49,7,6.06,BR,1619286562
215,lashio,22.9333,97.7500,72.03,49,9,1.30,MM,1619286782
229,el campo,29.1966,-96.2697,84.99,42,1,12.66,US,1619286989
258,karachi,24.9056,67.0822,86.00,42,0,8.34,PK,1619286995
271,cabo san lucas,22.8909,-109.9124,80.60,41,1,8.05,MX,1619286564
328,astorga,-23.2325,-51.6656,82.40,44,0,3.44,BR,1619287010
468,vallenar,-28.5708,-70.7581,76.08,33,0,6.55,CL,1619287039


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Find hotels in each city and place them into data frame
# Create data frame and reset index
hotel_df = ideal_cities
hotel_df = hotel_df.reset_index()
hotel_df = hotel_df.drop(columns="index")
# Set up a parameters dictionary
params = {"radius": 5000,
          "type": "lodging",
          "key": g_key}

# Base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # Get coordinates for city for each row, add to params dictionary
    lat = row["Lat"]
    lng = row["Lng"]
    coords = f"{lat},{lng}"
    params["location"] = coords

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response["results"]
    
    try:
        print(f"Closest hotel to {row['City']} ({coords}) is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Closest hotel to castro (-24.7911,-50.0119) is CHACARA BAILLY.
------------
Closest hotel to lashio (22.9333,97.75) is Hotel CF Lashio - Burmese Only.
------------
Closest hotel to el campo (29.1966,-96.2697) is Lonestar Inn, El Campo Texas.
------------
Closest hotel to karachi (24.9056,67.0822) is Sadaat.
------------
Closest hotel to cabo san lucas (22.8909,-109.9124) is Hotel Tesoro Los Cabos.
------------
Closest hotel to astorga (-23.2325,-51.6656) is Rancho Jads.
------------
Closest hotel to vallenar (-28.5708,-70.7581) is Humacao Bed & Breakfast.
------------


In [6]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,castro,-24.7911,-50.0119,73.76,49,7,6.06,BR,1619286562,CHACARA BAILLY
1,lashio,22.9333,97.7500,72.03,49,9,1.30,MM,1619286782,Hotel CF Lashio - Burmese Only
2,el campo,29.1966,-96.2697,84.99,42,1,12.66,US,1619286989,"Lonestar Inn, El Campo Texas"
3,karachi,24.9056,67.0822,86.00,42,0,8.34,PK,1619286995,Sadaat
4,cabo san lucas,22.8909,-109.9124,80.60,41,1,8.05,MX,1619286564,Hotel Tesoro Los Cabos
5,astorga,-23.2325,-51.6656,82.40,44,0,3.44,BR,1619287010,Rancho Jads
6,vallenar,-28.5708,-70.7581,76.08,33,0,6.55,CL,1619287039,Humacao Bed & Breakfast


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))